<a href="https://colab.research.google.com/github/Achmad96/anomaly-fraud-detection/blob/master/fraud_detection_on_transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install --upgrade tensorflow

In [2]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [3]:
from google.colab import drive
from pathlib import Path

drive_path = Path('/content/drive')
drive.mount(str(drive_path))
drive_path /= 'MyDrive'

Mounted at /content/drive


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
SEED = 42

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, auc, precision_recall_curve
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

In [76]:
dataset_root = drive_path / 'Fraud Dataset'

activities_df = pd.read_csv(dataset_root / "Customer Profiles/account_activity.csv")
customers_df = pd.read_csv(dataset_root / "Customer Profiles/customer_data.csv")
fraud_indicators_df = pd.read_csv(dataset_root / "Fraudulent Patterns/fraud_indicators.csv")
sus_activities_df = pd.read_csv(dataset_root / "Fraudulent Patterns/suspicious_activity.csv")
merchants_df = pd.read_csv(dataset_root / "Merchant Information/merchant_data.csv")
transaction_categories_df = pd.read_csv(dataset_root / "Merchant Information/transaction_category_labels.csv")
transaction_amounts_df = pd.read_csv(dataset_root / "Transaction Amounts/amount_data.csv")
anomaly_scores_df = pd.read_csv(dataset_root / "Transaction Amounts/anomaly_scores.csv")
transaction_metadata_df = pd.read_csv(dataset_root / "Transaction Data/transaction_metadata.csv")
transaction_records_df = pd.read_csv(dataset_root / "Transaction Data/transaction_records.csv")

In [77]:
df = pd.merge(activities_df, customers_df, on='CustomerID')
df = pd.merge(df, transaction_records_df, on='CustomerID')
df = pd.merge(df, sus_activities_df, on='CustomerID')
df = pd.merge(df, transaction_metadata_df, on='TransactionID')
df = pd.merge(df, anomaly_scores_df, on='TransactionID')
df = pd.merge(df, transaction_amounts_df, on='TransactionID')
df = pd.merge(df, transaction_categories_df, on='TransactionID')
df = pd.merge(df, fraud_indicators_df, on='TransactionID')
df = pd.merge(df, merchants_df, on='MerchantID')
df.head()

,CustomerID,AccountBalance,LastLogin,Name,Age,Address,TransactionID,Amount,SuspiciousFlag,Timestamp,MerchantID,AnomalyScore,TransactionAmount,Category,FraudIndicator,MerchantName,Location
0,1001,9507.272060,2022-01-01,Customer 1001,54,Address 1001,787,33.670626,0,2022-02-02 18:00:00,2627,0.030921,87.375444,Food,0,Merchant 2627,Location 2627
1,1003,1715.321989,2022-01-03,Customer 1003,40,Address 1003,641,30.980239,0,2022-01-27 16:00:00,2496,0.961552,46.044172,Travel,0,Merchant 2496,Location 2496
2,1004,3101.509134,2022-01-04,Customer 1004,30,Address 1004,450,11.167088,0,2022-01-19 17:00:00,2184,0.525809,85.085061,Other,0,Merchant 2184,Location 2184
3,1004,3101.509134,2022-01-04,Customer 1004,30,Address 1004,921,45.849784,0,2022-02-08 08:00:00,2111,0.652188,36.534720,Other,0,Merchant 2111,Location 2111
4,1005,5405.766914,2022-01-05,Customer 1005,46,Address 1005,599,81.808651,0,2022-01-25 22:00:00,2191,0.661493,82.852025,Other,0,Merchant 2191,Location 2191


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         1000 non-null   int64  
 1   AccountBalance     1000 non-null   float64
 2   LastLogin          1000 non-null   object 
 3   Name               1000 non-null   object 
 4   Age                1000 non-null   int64  
 5   Address            1000 non-null   object 
 6   TransactionID      1000 non-null   int64  
 7   Amount             1000 non-null   float64
 8   SuspiciousFlag     1000 non-null   int64  
 9   Timestamp          1000 non-null   object 
 10  MerchantID         1000 non-null   int64  
 11  AnomalyScore       1000 non-null   float64
 12  TransactionAmount  1000 non-null   float64
 13  Category           1000 non-null   object 
 14  FraudIndicator     1000 non-null   int64  
 15  MerchantName       1000 non-null   object 
 16  Location           1000 n

In [79]:
df.describe()

,CustomerID,AccountBalance,Age,TransactionID,Amount,SuspiciousFlag,MerchantID,AnomalyScore,TransactionAmount,FraudIndicator
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1496.626000,5715.455931,39.848000,500.500000,55.394871,0.025000,2504.701000,0.492282,55.853826,0.045000
std,289.229047,2540.517262,13.074932,288.819436,25.070809,0.156203,288.414883,0.288423,26.093606,0.207408
min,1001.000000,1056.301181,18.000000,1.000000,10.006933,0.000000,2001.000000,0.000234,10.057864,0.000000
25%,1239.000000,3489.554711,29.000000,250.750000,34.502315,0.000000,2254.500000,0.251802,33.875129,0.000000
50%,1501.000000,5753.007775,39.000000,500.500000,57.844325,0.000000,2500.500000,0.490242,55.958313,0.000000
75%,1739.250000,7925.710167,51.000000,750.250000,75.858576,0.000000,2761.250000,0.741888,77.589391,0.000000
max,2000.000000,9999.776239,64.000000,1000.000000,99.887410,1.000000,3000.000000,0.999047,99.784323,1.000000


In [80]:
suspicious_flags = df['SuspiciousFlag'].value_counts()
suspicious_flags_map = {
    0: 'Non-Suspicious',
    1: 'Suspicious'
}
suspicious_flags.index = suspicious_flags.index.map(suspicious_flags_map)
fig = px.bar(
    x=suspicious_flags.index,
    y=suspicious_flags.to_numpy(),
    labels={'x': 'Suspicious Flags', 'y': 'Count'},
    hover_name=suspicious_flags.index,
    color=suspicious_flags.index,
    color_discrete_sequence=px.colors.qualitative.T10)
fig.show()

In [106]:
fraud_indicators = df['FraudIndicator'].value_counts()
fraud_indicators_map = {
    0: 'Legit',
    1: 'Fraud'
}
fraud_indicators.index = fraud_indicators.index.map(fraud_indicators_map)
fig = px.bar(
    x=fraud_indicators.index,
    y=fraud_indicators.to_numpy(),
    labels={'x': 'Fraud Indicator', 'y': 'Count'},
    hover_name=fraud_indicators.index,
    color=fraud_indicators.index,
    color_discrete_sequence=px.colors.qualitative.T10,
    title='Class Distribution for Fraud Indicator')
fig.show()

In [82]:
df.columns

Index(['CustomerID', 'AccountBalance', 'LastLogin', 'Name', 'Age', 'Address',
       'TransactionID', 'Amount', 'SuspiciousFlag', 'Timestamp', 'MerchantID',
       'AnomalyScore', 'TransactionAmount', 'Category', 'FraudIndicator',
       'MerchantName', 'Location'],
      dtype='object')

In [83]:
numerical_features = df.select_dtypes(include=['int64', 'float64'])
categorical_features = df.select_dtypes(include=['object'])
print(f"""
Numerical Features:
{numerical_features.columns}

Categorical Features:
{categorical_features.columns}
""")


Numerical Features:
Index(['CustomerID', 'AccountBalance', 'Age', 'TransactionID', 'Amount',
       'SuspiciousFlag', 'MerchantID', 'AnomalyScore', 'TransactionAmount',
       'FraudIndicator'],
      dtype='object')

Categorical Features:
Index(['LastLogin', 'Name', 'Address', 'Timestamp', 'Category', 'MerchantName',
       'Location'],
      dtype='object')



In [84]:
drop_columns=['TransactionID','MerchantID','MerchantName','Location','CustomerID','Name', 'Age', 'Address']
df = df.drop(drop_columns, axis=1)

In [85]:
df.head()

,AccountBalance,LastLogin,Amount,SuspiciousFlag,Timestamp,AnomalyScore,TransactionAmount,Category,FraudIndicator
0,9507.272060,2022-01-01,33.670626,0,2022-02-02 18:00:00,0.030921,87.375444,Food,0
1,1715.321989,2022-01-03,30.980239,0,2022-01-27 16:00:00,0.961552,46.044172,Travel,0
2,3101.509134,2022-01-04,11.167088,0,2022-01-19 17:00:00,0.525809,85.085061,Other,0
3,3101.509134,2022-01-04,45.849784,0,2022-02-08 08:00:00,0.652188,36.534720,Other,0
4,5405.766914,2022-01-05,81.808651,0,2022-01-25 22:00:00,0.661493,82.852025,Other,0


In [86]:
if pd.api.types.is_datetime64_any_dtype(df['Timestamp']):
    print("The 'Timestamp' column is already in datetime format.")
else:
    print("The 'Timestamp' column is not in datetime format.")

The 'Timestamp' column is not in datetime format.


In [87]:
df['Datetime'] = pd.to_datetime(df['Timestamp'])

In [88]:
df['Hour'] = df['Datetime'].dt.hour
df['LastLogin'] = pd.to_datetime(df['LastLogin'])
df['Gap'] = (df['Datetime'] - df['LastLogin']).dt.days.abs()
df.head()

,AccountBalance,LastLogin,Amount,SuspiciousFlag,Timestamp,AnomalyScore,TransactionAmount,Category,FraudIndicator,Datetime,Hour,Gap
0,9507.272060,2022-01-01,33.670626,0,2022-02-02 18:00:00,0.030921,87.375444,Food,0,2022-02-02 18:00:00,18,32
1,1715.321989,2022-01-03,30.980239,0,2022-01-27 16:00:00,0.961552,46.044172,Travel,0,2022-01-27 16:00:00,16,24
2,3101.509134,2022-01-04,11.167088,0,2022-01-19 17:00:00,0.525809,85.085061,Other,0,2022-01-19 17:00:00,17,15
3,3101.509134,2022-01-04,45.849784,0,2022-02-08 08:00:00,0.652188,36.534720,Other,0,2022-02-08 08:00:00,8,35
4,5405.766914,2022-01-05,81.808651,0,2022-01-25 22:00:00,0.661493,82.852025,Other,0,2022-01-25 22:00:00,22,20


In [89]:
X = df.drop(['FraudIndicator','Timestamp','Datetime','LastLogin'],axis=1)
y = df['FraudIndicator']

In [90]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X['Category'] = label_encoder.fit_transform(X['Category'])
X.head()

,AccountBalance,Amount,SuspiciousFlag,AnomalyScore,TransactionAmount,Category,Hour,Gap
0,9507.272060,33.670626,0,0.030921,87.375444,0,18,32
1,1715.321989,30.980239,0,0.961552,46.044172,4,16,24
2,3101.509134,11.167088,0,0.525809,85.085061,2,17,15
3,3101.509134,45.849784,0,0.652188,36.534720,2,8,35
4,5405.766914,81.808651,0,0.661493,82.852025,2,22,20


In [91]:
X_encoded = X.copy()

In [92]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(random_state=SEED)

resampled_data = smote.fit_resample(X,y)
X_resampled = resampled_data[0]
y_resampled = resampled_data[1]

In [93]:
# from sklearn.utils import shuffle
# resampled = shuffle(X_resampled, y_resampled, random_state=SEED)
# X_resampled = resampled[0]
# y_resampled = resampled[1]

In [94]:
print(X.shape, y.shape, X_resampled.shape, y_resampled.shape)

(1000, 8) (1000,) (1910, 8) (1910,)


In [95]:
from collections import Counter
fraud_indicators = y_resampled.value_counts()
fraud_indicators_map = {
    0: 'Legit',
    1: 'Fraud'
}
fraud_indicators.index = fraud_indicators.index.map(fraud_indicators_map)
fig = px.bar(
    x=fraud_indicators.index,
    y=fraud_indicators.to_numpy(),
    labels={'x': 'Fraud Indicator', 'y': 'Count'},
    hover_name=fraud_indicators.index,
    color=fraud_indicators.index,
    title='Class Distribution after Oversampling',
    color_discrete_sequence=px.colors.qualitative.T10)
fig.show()

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=SEED)

In [97]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

model = DecisionTreeClassifier()

param_grid = {
    'max_depth': [None, 10, 25, 50,100,250],  # Maximum depth of the tree
    'min_samples_split': [2, 10, 50,100,250],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 10, 30,75,100],    # Minimum number of samples required to be at a leaf node
    'criterion': ['gini', 'entropy'] # Criterion to measure the quality of a split
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Estimators: ", grid_search.best_estimator_)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Estimators:  DecisionTreeClassifier(criterion='entropy')


In [98]:
best_params = grid_search.best_params_
best_model = DecisionTreeClassifier(**best_params)
best_model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [99]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print("Model Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)

Model Evaluation Metrics:
Accuracy: 0.8664921465968587
Precision: 0.8431372549019608
Recall: 0.900523560209424
F1 Score: 0.8708860759493671
Confusion Matrix:
[[159  32]
 [ 19 172]]


In [100]:
tn, fp, fn, tp = confusion.ravel()

precision_fraud = tp / (tp + fp)
print(f"Precision for Fraud class (Accuracy on Fraud only): {precision_fraud:.4f}")
print(f"\nInterpretation: This means that out of all transactions the model predicted as 'Fraud', approximately {precision_fraud:.2%} were actually fraudulent.")

Precision for Fraud class (Accuracy on Fraud only): 0.8431

Interpretation: This means that out of all transactions the model predicted as 'Fraud', approximately 84.31% were actually fraudulent.


In [101]:
y_pred = best_model.predict(X_test)
labels = ['Legit', 'Fraud']
crm = confusion_matrix(y_test, y_pred)
fig = px.imshow(
    crm,
    labels=dict(x="Predicted", y="Ground Truth", color="Count"),
    x=labels,
    y=labels,
    text_auto=True,
    color_continuous_scale="Blues")
fig.show()

In [102]:
df['Category'].value_counts()

,count
Category,
Other,210
Food,204
Travel,198
Online,196
Retail,192


In [103]:
import pandas as pd
import numpy as np

# test_data = df[df['FraudIndicator'] == 1].sample(5).reset_index(drop=True)
# ground_truth = test_data['FraudIndicator'].to_numpy()
# test_data = test_data.drop(['FraudIndicator','Timestamp','LastLogin','Datetime'], axis=1)

# INFERENCES DATA
test_data = {
    'AccountBalance': [10000.0, 450.0, 15000.0, 1200.0, 3500.0, 1200.0, 500.0, 45000.0, 3000.0],
    'Amount': [55.0, 400.0, 2500.0, 150.0, 25.0, 950.0, 480.0, 5000.0, 850.0],
    'SuspiciousFlag': [0, 1, 0, 1, 0, 1, 0, 0, 1],
    'AnomalyScore': [0.08, 0.99, 0.15, 0.92, 0.12, 0.98, 0.99, 0.12, 0.95],
    'TransactionAmount': [55.0, 400.0, 2500.0, 150.0, 25.0, 950.0, 480.0, 5000.0, 850.0],
    'Category': ['Food', 'Online', 'Travel', 'Online', 'Retail', 'Online', 'Travel', 'Retail', 'Online'],
    'Hour': [13, 3, 15, 2, 10, 3, 2, 14, 1],
    'Gap': [2, 850, 10, 500, 5, 5, 10, 2, 12]
}

# The ground truth for evaluation
# 0 = Legitimate, 1 = Fraud
ground_truth = [0, 1, 0, 1, 0, 1, 1, 0, 1]

test_df = pd.DataFrame(test_data)
test_df['Category'] = label_encoder.transform(test_df['Category'])
test_df.head()

,AccountBalance,Amount,SuspiciousFlag,AnomalyScore,TransactionAmount,Category,Hour,Gap
0,10000.0,55.0,0,0.08,55.0,0,13,2
1,450.0,400.0,1,0.99,400.0,1,3,850
2,15000.0,2500.0,0,0.15,2500.0,4,15,10
3,1200.0,150.0,1,0.92,150.0,1,2,500
4,3500.0,25.0,0,0.12,25.0,3,10,5


In [104]:
result = best_model.predict(test_df)
print("Test Dataframe with Ground Truth:")
print(test_df.head())
print("\nGround Truth:")
print(ground_truth)
print("\nPredictions:")
print(result)

Test Dataframe with Ground Truth:
   AccountBalance  Amount  SuspiciousFlag  AnomalyScore  TransactionAmount  \
0         10000.0    55.0               0          0.08               55.0   
1           450.0   400.0               1          0.99              400.0   
2         15000.0  2500.0               0          0.15             2500.0   
3          1200.0   150.0               1          0.92              150.0   
4          3500.0    25.0               0          0.12               25.0   

   Category  Hour  Gap  
0         0    13    2  
1         1     3  850  
2         4    15   10  
3         1     2  500  
4         3    10    5  

Ground Truth:
[0, 1, 0, 1, 0, 1, 1, 0, 1]

Predictions:
[0 0 0 0 0 0 0 0 0]


In [105]:
importances = best_model.feature_importances_
feature_names = X.columns

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
feature_importance_df['Importance'] = feature_importance_df['Importance'] * 100

fig = px.pie(
    feature_importance_df,
    names='Feature',
    values='Importance',
    title='Most Important Parameters Affecting the Accuracy'
)
fig.show()